In [1]:
from tools.prompt import generate_few_shot_ner_prompts

from tools.schema import Saude

from langchain_openai import ChatOpenAI

In [2]:
MODEL_NAME = "gpt-4o-mini"

extractor = ChatOpenAI(model=MODEL_NAME, timeout=180).with_structured_output(
    Saude, include_raw=False, method="function_calling"
)

In [3]:
EXAMPLE_TEXT = """Trata-se de pedido de tutela antecipada apresentado por Joana da Silva objetivando provimento jurisdicional que determine que a União, o Estado do Ceará e o Município de Fortaleza, de forma imediata e gratuita, realizem, em seu favor, exame médico de gastroenteroscopia, o qual se afigura necessário para garantir tratamento mais eficaz de provável doença de que padece. Salienta que esta na fila do SUS para a realização do referido exame desde 2011, não podendo mais aguardar em face do agravamento do seu estado de saúde (gastrite, osteoporose, diabetes e pressão alta), de tal sorte que o exame em questão é imprescindível para um melhor diagnóstico do seu problema físico. É o que interessa a relatar. Passo a DECIDIR. O instituto da antecipação da tutela, no plano geral do processo de cognição, nos termos do art. 273, caput e incisos, do CPC, é admissível quando da existência dos seguintes requisitos: i.) O Juiz, existindo prova inequívoca do fato, se convença da verossimilhança da alegação do autor; ii.) haja fundado receio de dano irreparável ou de difícil reparação ou fique caracterizado o abuso de direito de defesa ou o manifesto propósito protelatório do réu. O direito público subjetivo à saúde constitui uma prerrogativa jurídica indisponível assegurada, de forma assaz contundente, pela Constituição Federal, em seu art. 196, in verbis: Art. 196 – A saúde é direito de todos e dever do Estado, garantido mediante políticas sociais e econômicas que visem à redução do risco de doença e de outros agravos e ao acesso universal e igualitário às ações e serviços para sua promoção, proteção e recuperação. A saúde é, pois, um direito de todos e dever do estado lato sensu, que detém a obrigação de fornecer condições de seu pleno exercício, assegurado e disciplinado constitucionalmente, estando os entes federativos, solidariamente, obrigados a fornecer os medicamentos necessários àqueles que não possuem condições financeiras de adquiri-los, independente de protocolos e entraves burocráticos restritivos de direito, sob pena burla ao princípio da dignidade da pessoa humana. Na oportunidade, ressalto os princípios e diretrizes do Sistema Único de Saúde, constantes no art. 7o, I e II, da Lei no 8.080/90, ipsis litteris: Art. 7o – As ações e serviços públicos de saúde e os serviços privados contratados ou conveniados que integram o Sistema Único de Saúde (SUS) são desenvolvidos de acordo com as diretrizes previstas no art. 198 da Constituição Federal, obedecendo ainda aos seguintes princípios: I – universalidade de acesso aos serviços de saúde em todos os níveis de assistência; II – integralidade de assistência, entendida como conjunto articulado e contínuo das ações e serviços preventivos e curativos, individuais e coletivos, exigidos para cada caso em todos os níveis de complexidade do sistema.”(Grifei) A questão central, contudo, que deve ser respondida diz respeito aos critérios a serem adotados quando o jurisdicionado, ao ver recusado seu pedido pela Administração, procura guarida junto ao Poder Judiciário. Este Juízo tem adotado a posição no sentido de que, não obstante os pedidos sejam feitos em face da dramática urgência do medicamento, é necessária a adoção de variáveis que não podem deixar de ser observadas. Assim, dada a possibilidade de, casuisticamente, se verificarem as variáveis, deve o Judiciário acautelar-se no acolhimento do pleito, sob pena de conceder remédio a quem pode adquiri-lo, ou simplesmente pode ser apenas um medicamento suplementar, dispensável, ou ainda importado em substituição a similares existentes no País, quebrando assim o princípio legislativo do SUS de atendimento, em primeiro lugar, da população mais carente. No caso em apreço, a autora sofre de diabetes, hipertensão, osteoporose e diarréia crônica, necessitando da realização do exame de gastroenteroscopia para um melhor diagnóstico de sua enfermidade, conforme o laudo pericial anexado aos autos (v. anexo 19). In casu, o Perito do Juízo concluiu e expressamente firmou em seu parecer técnico que “.para investigação inicial de um quadro de diarréia crônica, a gastroenteroscopia deve ser um dos primeiros exames a serem realizados, juntamente com outros, como colonoscopia, trânsito intestinal, etc.”."""

In [4]:
prompt_with_few_shot = generate_few_shot_ner_prompts(EXAMPLE_TEXT)
prompt_with_few_shot

ChatPromptValue(messages=[SystemMessage(content='Você é um especialista em extração de entidades nomeadas com precisão excepcional. Sua tarefa é identificar e extrair informações específicas do texto fornecido, seguindo estas diretrizes:\n\n1. Extraia as informações exatamente como aparecem no texto, sem interpretações ou alterações.\n2. Se uma informação solicitada não estiver presente ou for ambígua, retorne null para esse campo.\n3. Mantenha-se estritamente dentro do escopo das entidades e atributos definidos no esquema fornecido.\n4. Preste atenção especial para manter a mesma ortografia, pontuação e formatação das informações extraídas.\n5. Não infira ou adicione informações que não estejam explicitamente presentes no texto.\n6. Se houver múltiplas menções da mesma entidade, extraia todas as ocorrências relevantes.\n7. Ignore informações irrelevantes ou fora do contexto das entidades solicitadas.\n\nLembre-se: sua precisão e aderência ao texto original são cruciais para o sucesso 

In [5]:
result = extractor.invoke(prompt_with_few_shot)

In [6]:
result.model_dump()

{'lista_medicamentos': [],
 'lista_doencas': [{'nome': 'gastrite', 'cid': None},
  {'nome': 'osteoporose', 'cid': None},
  {'nome': 'diabetes', 'cid': None},
  {'nome': 'pressão alta', 'cid': None},
  {'nome': 'diarreia crônica', 'cid': None}],
 'lista_suplementos': [],
 'lista_procedimentos': [{'nome': 'gastroenteroscopia',
   'tipo': 'diagnóstico'}],
 'lista_insumos': [],
 'lista_pessoas': [{'nome': 'Joana da Silva', 'funcao_processual': 'autor'}]}